In [25]:
import pandas as pd

import lightgbm as lgb
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error


In [26]:
data = pd.read_csv("Old data/synop_all_nan_removed.csv",index_col=0)

In [27]:
data

,to,stationId,latitude,longitude,max_temp_w_date,max_wind_speed_10min,max_wind_speed_3sec,mean_pressure,mean_relative_hum,mean_temp,mean_wind_dir,mean_wind_speed,min_temp,vapour_pressure_deficit_mean
2,2011-01-01T00:00:00+00:00,6032,57.3828,10.3349,4.0,14.1,19.2,997.3,75.5,3.7,282.0,13.0,3.2,0.2
3,2011-01-01T00:00:00+00:00,6041,57.7364,10.6316,4.8,12.7,17.7,995.3,68.5,4.6,287.0,12.1,4.4,0.3
4,2011-01-01T00:00:00+00:00,6049,56.5604,10.0929,4.0,18.7,27.4,1001.3,89.2,3.6,287.0,15.1,3.0,0.1
6,2011-01-01T00:00:00+00:00,6052,56.7068,8.2150,4.7,15.1,20.7,1005.9,87.3,4.2,302.0,13.6,4.1,0.1
7,2011-01-01T00:00:00+00:00,6056,56.3830,8.6705,5.1,12.8,24.9,1005.6,93.8,4.8,284.0,11.8,4.7,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5447322,2022-05-12T17:00:00+00:00,6181,55.7664,12.5263,14.9,6.3,12.3,1012.4,54.2,14.5,274.0,5.4,14.2,0.8
5447325,2022-05-12T17:00:00+00:00,6188,55.8764,12.4121,14.5,9.2,14.1,1012.2,56.5,13.8,263.0,7.9,13.4,0.7
5447326,2022-05-12T17:00:00+00:00,6190,55.0677,14.7494,12.4,7.2,9.3,1013.7,80.8,12.1,252.0,6.4,11.7,0.3
5447327,2022-05-12T17:00:00+00:00,6193,55.2979,14.7718,12.3,10.2,11.9,1013.0,79.2,11.6,237.0,9.0,11.1,0.3


In [28]:
data["to"].unique()

array(['2011-01-01T00:00:00+00:00', '2011-01-01T01:00:00+00:00',
       '2011-01-01T02:00:00+00:00', ..., '2022-05-12T15:00:00+00:00',
       '2022-05-12T16:00:00+00:00', '2022-05-12T17:00:00+00:00'],
      dtype=object)

In [29]:
data["stationId"].value_counts()


6032    99548
6156    99548
6124    99548
6126    99548
6135    99548
6138    99548
6141    99548
6147    99548
6149    99548
6151    99548
6154    99548
6159    99548
6041    99548
6168    99548
6169    99548
6170    99548
6174    99548
6180    99548
6181    99548
6188    99548
6190    99548
6193    99548
6123    99548
6197    99548
6119    99548
6081    99548
6049    99548
6056    99548
6058    99548
6065    99548
6068    99548
6072    99548
6073    99548
6079    99548
6102    99548
6116    99548
6080    99547
6118    99547
6096    99547
6074    99546
6104    99546
6070    99542
6120    99542
6052    99508
6093    99499
6108    67214
6136    21335
Name: stationId, dtype: int64

In [30]:
idx_list = data["stationId"].value_counts().index.to_list()
print(idx_list)

[6032, 6156, 6124, 6126, 6135, 6138, 6141, 6147, 6149, 6151, 6154, 6159, 6041, 6168, 6169, 6170, 6174, 6180, 6181, 6188, 6190, 6193, 6123, 6197, 6119, 6081, 6049, 6056, 6058, 6065, 6068, 6072, 6073, 6079, 6102, 6116, 6080, 6118, 6096, 6074, 6104, 6070, 6120, 6052, 6093, 6108, 6136]


In [31]:
idx_list[-11:]

[6080, 6118, 6096, 6074, 6104, 6070, 6120, 6052, 6093, 6108, 6136]

In [32]:
for i in range(1,12):
    data.drop(data[data['stationId']==idx_list[-i]].index,inplace=True)



In [33]:
data

,to,stationId,latitude,longitude,max_temp_w_date,max_wind_speed_10min,max_wind_speed_3sec,mean_pressure,mean_relative_hum,mean_temp,mean_wind_dir,mean_wind_speed,min_temp,vapour_pressure_deficit_mean
2,2011-01-01T00:00:00+00:00,6032,57.3828,10.3349,4.0,14.1,19.2,997.3,75.5,3.7,282.0,13.0,3.2,0.2
3,2011-01-01T00:00:00+00:00,6041,57.7364,10.6316,4.8,12.7,17.7,995.3,68.5,4.6,287.0,12.1,4.4,0.3
4,2011-01-01T00:00:00+00:00,6049,56.5604,10.0929,4.0,18.7,27.4,1001.3,89.2,3.6,287.0,15.1,3.0,0.1
7,2011-01-01T00:00:00+00:00,6056,56.3830,8.6705,5.1,12.8,24.9,1005.6,93.8,4.8,284.0,11.8,4.7,0.1
8,2011-01-01T00:00:00+00:00,6058,56.0072,8.1413,4.6,14.5,19.4,1008.1,98.0,4.3,285.0,13.8,4.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5447322,2022-05-12T17:00:00+00:00,6181,55.7664,12.5263,14.9,6.3,12.3,1012.4,54.2,14.5,274.0,5.4,14.2,0.8
5447325,2022-05-12T17:00:00+00:00,6188,55.8764,12.4121,14.5,9.2,14.1,1012.2,56.5,13.8,263.0,7.9,13.4,0.7
5447326,2022-05-12T17:00:00+00:00,6190,55.0677,14.7494,12.4,7.2,9.3,1013.7,80.8,12.1,252.0,6.4,11.7,0.3
5447327,2022-05-12T17:00:00+00:00,6193,55.2979,14.7718,12.3,10.2,11.9,1013.0,79.2,11.6,237.0,9.0,11.1,0.3


In [34]:
data["stationId"].value_counts()

6032    99548
6041    99548
6147    99548
6149    99548
6151    99548
6154    99548
6156    99548
6159    99548
6168    99548
6169    99548
6170    99548
6174    99548
6180    99548
6181    99548
6188    99548
6190    99548
6193    99548
6141    99548
6138    99548
6135    99548
6073    99548
6049    99548
6056    99548
6058    99548
6065    99548
6068    99548
6072    99548
6079    99548
6126    99548
6081    99548
6102    99548
6116    99548
6119    99548
6123    99548
6124    99548
6197    99548
Name: stationId, dtype: int64

In [35]:
data.to_csv("data_clean.csv",index=False)

def make_graph(df,time_index,k_neighbours=5,print_output=True):
    """
    Makes a Data object for use with GNN in torch_geometric from a dataframe of synop DMI stations.
    :param df: Dataframe of stations at all timesteps.
    :param time_index: Index of the timestep in df["to"].unique().
    :param k_neighbours: Number of neighbours to connect the graph to.
    :param print_output: Print the timestep, number of stations, and list of stations.
    :return: Data object for GNN in torch_geometric.
    """
    timestep = df["to"].unique()[time_index]
    df_middle = df[df["to"] == timestep]
    df_timestep = df_middle.drop("to",axis=1)
    x = []
    x_coords = []

    for i in range(len(df_timestep["stationId"].unique())):
        station = df_timestep.iloc[i]
        x.append(station)
        x_coords.append((station[1:3]))

    if print_output==True:
        print(f"Stations at timestep: {timestep}")
        print(f"Number of stations: {len(df_timestep['stationId'].unique())}")
        print(f"Stations included: {df_timestep['stationId'].unique()}")

    edges = torch_geometric.nn.knn_graph(x_coords,k_neighbours)
    x_tensor = torch.tensor(x,dtype=torch.float)
    data = torch_geometric.data.Data(x=x, edge_index=edges)

    return data

In [19]:
data_6190 = data[data["stationId"]==6190]
data_6193 = data[data["stationId"]==6193]
data_6197 = data[data["stationId"]==6197]

In [20]:
data_6190.to_csv("station_6190_timestamp.csv",index=False)
data_6193.to_csv("station_6193_timestamp.csv",index=False)
data_6197.to_csv("station_6197_timestamp.csv",index=False)